In [2]:
pip install PyPortfolioOpt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 7.6 MB/s eta 0:00:00


In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp
from pypfopt import EfficientFrontier, risk_models, expected_returns, EfficientCVaR

In [4]:
tickers = [
    "AAPL", "MSFT", "GOOG", "BRK-B", "JPM", "JNJ",
    "V", "PG", "UNH", "HD", "MA", "BAC", "VZ",
    "ADBE", "CMCSA", "T", "INTC", "CSCO", "PFE", "CRM", "ABT", "TMO",
    "CVX", "MRK", "ORCL", "KO", "ACN", "MCD", "NKE", "WMT", "NEE",
    "IBM", "TXN", "COST", "LIN", "SBUX", "PM", "LOW", "QCOM", "BMY"
]

riskfree = 0.05

# Download historical adjusted close prices (2020-2025)
data = yf.download(tickers, start="2015-01-01", end="2025-01-01")["Close"]
data = data.dropna(axis=1)  # Remove any tickers with missing data

# Split data into training (in-sample) and testing (out-of-sample) periods.
# In-sample: 2020-01-01 to 2023-01-01; Out-of-sample: 2023-01-02 to 2025-01-01
train_data = data.loc["2015-01-01":"2023-01-01"]
test_data = data.loc["2023-01-02":"2025-01-01"]

# Calculate expected returns and covariance matrix from the training data.
mu_train = expected_returns.mean_historical_return(train_data)
S_train = risk_models.sample_cov(train_data)

with open('tickers.txt', 'w') as f:
    for ticker in tickers:
        f.write(ticker + '\n')

[*********************100%***********************]  40 of 40 completed


In [5]:
#For the efficientFrontier function in the package pypfopt (see for more additional information: https://pyportfolioopt.readthedocs.io/en/latest/MeanVariance.html?highlight=efficientfrontier)
#In the documentation mu = expected returns, S = cov_matrix, weight_bounds = the minimum and maximum weight of each asset OR single min/max pair if all identical, defaults to (0, 1). Must be changed to (-1, 1) for portfolios with shorting.

ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))

#The function ef.max_sharpe(risk_free_rate=riskfree) finds the maximum sharpe ratio
objective == 'max_sharpe':
weights = ef.max_sharpe(risk_free_rate=riskfree)
ret_train, vol_train, sr_train = ef.portfolio_performance(verbose=False, risk_free_rate=riskfree)

#For the EfficientCVaR (See for more information: https://pyportfolioopt.readthedocs.io/en/latest/GeneralEfficientFrontier.html?highlight=efficientcvar#efficient-cvar)
#In the documentation mu = expected returns, train_data = returns
#The function efficient_risk(target_cvar) = Maximise return for a target CVaR
ef = EfficientCVaR(mu, train_data, beta=0.95, weight_bounds=(0, 1))
weights = ef.efficient_risk(maxcvar)
ret_train, vol_train = ef.portfolio_performance(verbose=False)

In [ ]:
#To add regularisation please see the section "Adding objectives and constraints" at https://pyportfolioopt.readthedocs.io/en/latest/MeanVariance.html?highlight=reg#adding-objectives-and-constraints
ef.add_objective(lambda w: l2_reg * cp.sum_squares(w))